# HuggingFace 커스텀 프로젝트 만들기 [프로젝트]

## 프로젝트 : 커스텀 프로젝트 직접 만들기


실습 코드에서 수행해 본 내용을 토대로, 이번에는 한국어 데이터셋에 도전해보겠습니다.

앞서 본 GLUE benchmark의 한국어 버전 [KLUE benchmark](https://klue-benchmark.com/)를 들어보신 적 있나요?

GLUE와 마찬가지로 한국어 자연어처리에 대한 이해도를 높이기 위해 만들어진 데이터셋 benchmark입니다. 총 8가지의 데이터셋이 있습니다. 다만 이번 시간에 진행할 프로젝트는 KLUE의 dataset을 활용하는 것이 아닌, model(klue/ber-base)를 활용하여 NSMC(Naver Sentiment Movie Corpus) task를 도전해보겠습니다.

모델과 데이터에 관한 정보는 링크를 참조해주세요.

- KLUE/Bert-base: https://huggingface.co/klue/bert-base
- NSMC: https://github.com/e9t/nsmc




### 라이브러리

In [1]:
import tensorflow
import numpy
import transformers
import datasets

print(tensorflow.__version__)
print(numpy.__version__)
print(transformers.__version__)
print(datasets.__version__)

2.6.0
1.21.4
4.11.3
1.14.0


### NSMC 데이터 분석 및 Huggingface dataset 구성


In [3]:
!pip install --upgrade datasets huggingface_hub fsspec

  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)


In [2]:
import datasets
from datasets import load_dataset

nsmc_dataset = load_dataset("e9t/nsmc")
print(nsmc_dataset)

Using custom data configuration default
Reusing dataset nsmc (/aiffel/.cache/huggingface/datasets/e9t___nsmc)/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})


In [3]:
# 데이터 확인

for i in range(5):
    for col in nsmc_dataset['train'].column_names:
        print(col,":",nsmc_dataset['train'][col][i])
    print("\n")

id : 9976970
document : 아 더빙.. 진짜 짜증나네요 목소리
label : 0


id : 3819312
document : 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
label : 1


id : 10265843
document : 너무재밓었다그래서보는것을추천한다
label : 0


id : 9045019
document : 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
label : 0


id : 6483659
document : 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
label : 1




### klue/bert-base model 및 tokenizer 불러오기

In [4]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base", num_labels=2)

# model = AutoModel.from_pretrained("klue/bert-base")
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")


Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

### 위에서 불러온 tokenizer으로 데이터셋을 전처리하고, model 학습 진행해 보기

In [5]:
def transform(data):
    return tokenizer(
        data['document'],
        truncation = True,
        padding = 'max_length',
        return_token_type_ids = False,
        )

In [6]:
nsmc_dataset_tokenized = nsmc_dataset.map(transform,batched=True)
nsmc_dataset_tokenized

Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/e9t___nsmc)/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-e348f78ab3e01d51.arrow
Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/e9t___nsmc)/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-9319fc860318ae2d.arrow


DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'document', 'id', 'input_ids', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['attention_mask', 'document', 'id', 'input_ids', 'label'],
        num_rows: 50000
    })
})

In [7]:
nsmc_train = nsmc_dataset_tokenized['train']
nsmc_test = nsmc_dataset_tokenized['test']

#### validation set split

모델 학습을 위해 validation set을 나눔

In [8]:
split_dataset = nsmc_train.train_test_split(test_size=0.2, seed=42)
train = split_dataset["train"]
valid = split_dataset["test"]

Loading cached split indices for dataset at /aiffel/.cache/huggingface/datasets/e9t___nsmc)/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-e9e1fef60353c2a0.arrow and /aiffel/.cache/huggingface/datasets/e9t___nsmc)/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-5be70b002d95d50d.arrow


In [9]:
train

Dataset({
    features: ['attention_mask', 'document', 'id', 'input_ids', 'label'],
    num_rows: 120000
})

In [10]:
valid

Dataset({
    features: ['attention_mask', 'document', 'id', 'input_ids', 'label'],
    num_rows: 30000
})

In [11]:
test = nsmc_test

In [12]:
test

Dataset({
    features: ['attention_mask', 'document', 'id', 'input_ids', 'label'],
    num_rows: 50000
})

In [ ]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments

output_dir = './'

training_arguments = TrainingArguments(
    output_dir,                        # output이 저장될 경로
    eval_strategy="epoch",             #evaluation하는 빈도
    learning_rate = 2e-5,              #learning_rate
    per_device_train_batch_size = 8,   # 각 device 당 batch size
    per_device_eval_batch_size = 8,    # evaluation 시에 batch size
    num_train_epochs = 3,              # train 시킬 총 epochs
    weight_decay = 0.01,               # weight decay: 가중치에 페널티를 주는 방식의 L2 정규화
)

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.6 MB/s eta 0:00:00


In [ ]:
# mrpc와 같은 이중분류 문제이므로, mrpc metric을 그대로 사용한다. 만약, task가 달라지면 다른 metric지표를 사용해야 한다.

# from datasets import load_metric
# metric = load_metric('glue', 'mrpc')

import evaluate
metric = evaluate.load('glue', 'mrpc')

def compute_metrics(eval_pred):
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels)

In [ ]:
trainer = Trainer(
    model=model,           # 학습시킬 model
    args=training_arguments,           # TrainingArguments을 통해 설정한 arguments
    train_dataset=train,    # training dataset
    eval_dataset=valid,       # evaluation dataset
    compute_metrics=compute_metrics,
)
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: years-archive (huggingface-helper) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.294200,0.281919,0.897333,0.896512
2,0.229600,0.366996,0.903233,0.904093
3,0.141200,0.467617,0.903333,0.903423


TrainOutput(global_step=45000, training_loss=0.23806507788764106, metrics={'train_runtime': 8663.6828, 'train_samples_per_second': 41.553, 'train_steps_per_second': 5.194, 'total_flos': 9.47199799296e+16, 'train_loss': 0.23806507788764106, 'epoch': 3.0})

In [ ]:
trainer.evaluate(test)

{'eval_loss': 0.47124406695365906,
 'eval_accuracy': 0.90182,
 'eval_f1': 0.9031277750370005,
 'eval_runtime': 352.2722,
 'eval_samples_per_second': 141.936,
 'eval_steps_per_second': 17.742,
 'epoch': 3.0}

    - eval_strategy="epoch",
    - learning_rate = 2e-5,
    - per_device_train_batch_size = 8,
    - per_device_eval_batch_size = 8,
    - num_train_epochs = 3,
    - weight_decay = 0.01,

    이 파라미터에 대해서 정확도 90% 나왔다.

### Fine-tuning을 통하여 모델 성능(accuarcy) 향상시키기

데이터 전처리, TrainingArguments 등을 조정하여 모델의 정확도를 90% 이상으로 끌어올려봅시다.


##### batch_size = 16

In [ ]:
another_model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
output_dir = "./another_model"

training_arguments = TrainingArguments(
    output_dir,                        # output이 저장될 경로
    eval_strategy="epoch",             # evaluation하는 빈도
    learning_rate = 2e-5,              # learning_rate
    per_device_train_batch_size = 16,   # 각 device 당 batch size
    per_device_eval_batch_size = 16,    # evaluation 시에 batch size
    num_train_epochs = 3,              # train 시킬 총 epochs
    weight_decay = 0.01,               # weight decay: 가중치에 페널티를 주는 방식의 L2 정규화
)

In [ ]:
trainer = Trainer(
    model=another_model,               # 학습시킬 model
    args=training_arguments,           # TrainingArguments을 통해 설정한 arguments
    train_dataset=train,               # training dataset
    eval_dataset=valid,                # evaluation dataset
    compute_metrics=compute_metrics,
)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.255300,0.245389,0.901700,0.901163
2,0.190700,0.288252,0.903267,0.904508
3,0.116600,0.385049,0.905400,0.905664


TrainOutput(global_step=22500, training_loss=0.19990860341389974, metrics={'train_runtime': 7795.4816, 'train_samples_per_second': 46.181, 'train_steps_per_second': 2.886, 'total_flos': 9.47199799296e+16, 'train_loss': 0.19990860341389974, 'epoch': 3.0})

In [ ]:
trainer.evaluate(test)

{'eval_loss': 0.38770389556884766,
 'eval_accuracy': 0.90318,
 'eval_f1': 0.9046127169907983,
 'eval_runtime': 331.0277,
 'eval_samples_per_second': 151.045,
 'eval_steps_per_second': 9.44,
 'epoch': 3.0}

In [ ]:
!rm -rf ./checkpoint-*

##### batch_size = 32

In [ ]:
third_model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!mkdir ./third_model

In [ ]:
output_dir = "./third_model"

training_arguments = TrainingArguments(
    output_dir,                        # output이 저장될 경로
    eval_strategy="epoch",             # evaluation하는 빈도
    learning_rate = 2e-5,              # learning_rate
    per_device_train_batch_size = 32,   # 각 device 당 batch size
    per_device_eval_batch_size = 32,    # evaluation 시에 batch size
    num_train_epochs = 3,              # train 시킬 총 epochs
    weight_decay = 0.01,               # weight decay: 가중치에 페널티를 주는 방식의 L2 정규화
    save_total_limit=3,                # checkpoint 최신 3개만 유지
)

In [ ]:
trainer = Trainer(
    model=third_model,               # 학습시킬 model
    args=training_arguments,           # TrainingArguments을 통해 설정한 arguments
    train_dataset=train,               # training dataset
    eval_dataset=valid,                # evaluation dataset
    compute_metrics=compute_metrics,
)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.244900,0.243496,0.900833,0.899313
2,0.172600,0.260214,0.904900,0.905595
3,0.121100,0.316923,0.905067,0.905180


TrainOutput(global_step=11250, training_loss=0.19227848442925347, metrics={'train_runtime': 7353.4833, 'train_samples_per_second': 48.956, 'train_steps_per_second': 1.53, 'total_flos': 9.47199799296e+16, 'train_loss': 0.19227848442925347, 'epoch': 3.0})

In [ ]:
trainer.evaluate(test)

{'eval_loss': 0.3163806200027466,
 'eval_accuracy': 0.90344,
 'eval_f1': 0.9046528161782131,
 'eval_runtime': 313.3749,
 'eval_samples_per_second': 159.553,
 'eval_steps_per_second': 4.988,
 'epoch': 3.0}

In [ ]:
!rm -rf ./another_model

##### batch_size=8, weight_decat=0.1

In [ ]:
fourth_model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!mkdir ./fourth_model

In [ ]:
output_dir = "./fourth_model"

training_arguments = TrainingArguments(
    output_dir,                        # output이 저장될 경로
    eval_strategy="epoch",             # evaluation하는 빈도
    learning_rate = 2e-5,              # learning_rate
    per_device_train_batch_size = 8,   # 각 device 당 batch size
    per_device_eval_batch_size = 8,    # evaluation 시에 batch size
    num_train_epochs = 3,              # train 시킬 총 epochs
    weight_decay = 0.1,               # weight decay: 가중치에 페널티를 주는 방식의 L2 정규화
    save_total_limit=3,                # checkpoint 최신 3개만 유지
)

In [ ]:
trainer = Trainer(
    model=fourth_model,               # 학습시킬 model
    args=training_arguments,           # TrainingArguments을 통해 설정한 arguments
    train_dataset=train,               # training dataset
    eval_dataset=valid,                # evaluation dataset
    compute_metrics=compute_metrics,
)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.294400,0.277378,0.899200,0.897824
2,0.231300,0.369901,0.901833,0.902796
3,0.138400,0.477315,0.902800,0.902729


TrainOutput(global_step=45000, training_loss=0.2339527097913954, metrics={'train_runtime': 8554.9858, 'train_samples_per_second': 42.081, 'train_steps_per_second': 5.26, 'total_flos': 9.47199799296e+16, 'train_loss': 0.2339527097913954, 'epoch': 3.0})

In [ ]:
trainer.evaluate(test)

{'eval_loss': 0.479248970746994,
 'eval_accuracy': 0.9019,
 'eval_f1': 0.9031876048554228,
 'eval_runtime': 348.5741,
 'eval_samples_per_second': 143.442,
 'eval_steps_per_second': 17.93,
 'epoch': 3.0}

### Bucketing을 적용하여 학습시키고, STEP 4의 결과와의 비교

아래 링크를 바탕으로 bucketing과 dynamic padding이 무엇인지 알아보고, 이들을 적용하여 model을 학습시킵니다.

- Data Collator: https://huggingface.co/docs/transformers/v4.30.0/en/main_classes/data_collator

- Trainer.TrainingArguments 의 group_by_length: https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments

STEP 4에 학습한 결과와 bucketing을 적용하여 학습시킨 결과를 비교해보고, 모델 성능 향상과 훈련 시간 두 가지 측면에서 각각 어떤 이점이 있는지 비교해봅시다.

In [12]:
!mkdir ./results

colab gpu 소진으로 노드에서 수행

In [15]:
fifth_model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base", num_labels=2)

loading configuration file https://huggingface.co/klue/bert-base/resolve/main/config.json from cache at /aiffel/.cache/huggingface/transformers/fbd0b2ef898c4653902683fea8cc0dd99bf43f0e082645b913cda3b92429d1bb.99b3298ed554f2ad731c27cdb11a6215f39b90bc845ff5ce709bb4e74ba45621
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading weights file https://huggingface.co/klue/bert-base/resolve/main/pytorch_model.bin from cache at /aiffel/.cache/huggingface/transform

In [16]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)  # dynamic padding 적용

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",         # 매 epoch 저장
    save_total_limit=2,  
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    group_by_length=True,  # 🔥 Bucketing 핵심 옵션
)

trainer = Trainer(
    model=fifth_model,
    args=training_args,
    train_dataset=train,
    eval_dataset=valid,
    tokenizer=tokenizer,
    data_collator=data_collator  # dynamic padding
)

trainer.train()


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, document.
***** Running training *****
  Num examples = 120000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 45000
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss
1,0.312400,0.316231
2,0.241700,0.370017


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, document.
***** Running Evaluation *****
  Num examples = 30000
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-15000
Configuration saved in ./results/checkpoint-15000/config.json
Model weights saved in ./results/checkpoint-15000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-15000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-15000/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, document.
***** Running Evaluation *****
  Num examples = 30000
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-30000
Configuration saved in ./results/checkpoint-30000/config.json
Model weights saved in ./results/checkpoint-30000/pytorch_model.bin
tokenize

KeyboardInterrupt: 

In [ ]:
trainer.evaluate(test)

In [ ]:
tokenizer.model_max_length

# 회고

Test set에 대한 결과 비교

| 구분 | loss | accuracy | f1 |
| --- | --- | --- | --- |
|bsz = 8, wd = 0.01 | 0.4712 | 0.9018 | 0.9031 |
|bsz = 16, wd = 0.01 | 0.3877 | 0.9032 | 0.9046 |
|bsz = 32, wd = 0.01 | 0.3163 | 0.9034 | 0.9047 |
|bsz = 8, wd = 0.1   | 0.4792 | 0.9019 | 0.9032 |




첫 모델을 학습시킬때, 학습시간이 10시간이 넘어서 코랩으로 진행했었는데, 코랩에서 gpu 소진되어 bucketing 진행은 다시 노드로 진행했다.   
bucketing 또한 11시간 넘게 걸려서 속도가 더 좋아졌다고 판단하기 어렵다. 길이 순서대로 정렬하는데 더 시간이 소요되는 것 같다.  
데이터셋을 만들때 미리 bucketing을 진행하고 해보면 어떻게 달라질지 궁금하다.  

checkpoint 전략이 500 step 마다 였는데, save_strategy = "epoch"으로 변경하니 1시간 정도 예상시간이 줄었다.  

colab으로 진행했을때, batch_size를 늘리면 속도는 줄었지만, 성능차이는 거의 보이지 않았다.  


### result archive




##### baseline model


- eval_strategy="epoch",             #evaluation하는 빈도
- learning_rate = 2e-5,              #learning_rate
- per_device_train_batch_size = 8,   # 각 device 당 batch size
- per_device_eval_batch_size = 8,    # evaluation 시에 batch size
- num_train_epochs = 3,              # train 시킬 총 epochs
- weight_decay = 0.01,    

- 구글 코랩 A100 GPU

Train  
[45000/45000 2:22:06, Epoch 3/3]

| Epoch |	Training Loss |	Validation Loss | 	Accuracy |	F1 |
| ------ | --------------- | ---------------- | --------- | ---- |
| 1 |	0.294200 |	0.281919 |	0.897333 |	0.896512 |
| 2 |	0.229600 |	0.366996 |	0.903233 |	0.904093 |
| 3 |	0.141200 |	0.467617 |	0.903333 |	0.903423 |

Test   
[6250/6250 05:52]
```
{'eval_loss': 0.47124406695365906,
 'eval_accuracy': 0.90182,
 'eval_f1': 0.9031277750370005,
 'eval_runtime': 352.2722,
 'eval_samples_per_second': 141.936,
 'eval_steps_per_second': 17.742,
 'epoch': 3.0}
```

---
##### batch size = 16

Train  
[22500/22500 2:09:55, Epoch 3/3]

| Epoch |	Training Loss |	Validation Loss |	Accuracy |	F1 |
| ------ | --------------- | ---------------- | --------- | ---- |
| 1 |	0.255300 |	0.245389 |	0.901700 |	0.901163 |
| 2 |	0.190700 |	0.288252 |	0.903267 |	0.904508 |
| 3 |	0.116600 |	0.385049 |	0.905400 |	0.905664 |

Test
[3125/3125 05:30]

```
{'eval_loss': 0.38770389556884766,
 'eval_accuracy': 0.90318,
 'eval_f1': 0.9046127169907983,
 'eval_runtime': 331.0277,
 'eval_samples_per_second': 151.045,
 'eval_steps_per_second': 9.44,
 'epoch': 3.0}
```

---

##### batch size = 32

Train  
[11250/11250 2:02:32, Epoch 3/3]

| Epoch |	Training Loss |	Validation Loss |	Accuracy |	F1 |
| --- | --- | --- | --- | --- |
| 1 |	0.244900 |	0.243496 |	0.900833 |	0.899313 |
| 2 |	0.172600 |	0.260214 |	0.904900 |	0.905595 |
| 3 |	0.121100 |	0.316923 |	0.905067 |	0.905180 |

Test
[1563/1563 05:12]

```
{'eval_loss': 0.3163806200027466,
 'eval_accuracy': 0.90344,
 'eval_f1': 0.9046528161782131,
 'eval_runtime': 313.3749,
 'eval_samples_per_second': 159.553,
 'eval_steps_per_second': 4.988,
 'epoch': 3.0}
```

##### batch_size=8, weight_decat=0.1

Train  
[45000/45000 2:22:34, Epoch 3/3]


| Epoch |	Training Loss |	Validation Loss |	Accuracy |	F1 |
| --- | --- | --- | --- | --- |
| 1 |	0.294400 |	0.277378 |	0.899200 |	0.897824 |
| 2 |	0.231300 |	0.369901 |	0.901833 |	0.902796 |
| 3 |	0.138400 |	0.477315 |	0.902800 |	0.902729 |

Test  
[6250/6250 05:48]
```
{'eval_loss': 0.479248970746994,
 'eval_accuracy': 0.9019,
 'eval_f1': 0.9031876048554228,
 'eval_runtime': 348.5741,
 'eval_samples_per_second': 143.442,
 'eval_steps_per_second': 17.93,
 'epoch': 3.0}
```

## bucketing & dynamic padding 개념이해



좋은 질문입니다! `bucketing`과 `dynamic padding`은 자연어 처리(NLP) 모델 학습에서 **효율적인 배치 처리(batch processing)** 를 위해 사용되는 **텍스트 길이 최적화 기법**입니다. 아래에서 개념과 차이를 쉽게 설명드릴게요.

---

### ✅ 1. Dynamic Padding (동적 패딩)

#### 📌 개념:

* **동적(dynamic)** 으로, **각 배치 내에서 가장 긴 문장 길이에 맞춰 padding**을 하는 방식입니다.
* → 전체 데이터셋에서 가장 긴 문장에 맞춰 패딩하지 않기 때문에 **메모리 낭비가 적습니다.**

#### ✅ 예시:

```python
Batch 1: ["나는 밥을 먹었다", "배고파"]
→ 토큰 길이: [5, 3] → max_len = 5 → [5, 5]로 패딩

Batch 2: ["오전에는 공부하고", "저녁에 운동함", "그리고 잠"]
→ 토큰 길이: [5, 4, 3] → max_len = 5 → [5, 5, 5]로 패딩
```

#### ✅ 장점:

* GPU 메모리 사용이 최적화됨
* 전체 max length로 패딩하는 것보다 연산량 ↓

---

### ✅ 2. Bucketing (버킷팅)

#### 📌 개념:

* **유사한 길이의 문장들끼리 묶어서(mini-batch로)** 학습하는 방식입니다.
* → 패딩이 거의 필요 없도록 **비슷한 길이의 문장들만 묶기 때문에 효율적입니다.**

#### ✅ 작동 방식:

1. 전체 데이터를 문장 길이에 따라 정렬
2. 비슷한 길이의 데이터끼리 묶어 "버킷(bucket)" 생성
3. 각 버킷에서 배치를 구성하여 학습

#### ✅ 예시:

```plaintext
길이별로 정렬 후
[3, 3, 4] → Batch 1
[10, 9, 11] → Batch 2
```

#### ✅ 장점:

* padding이 거의 필요 없음
* 연산 자원 절약

---

### ✅ 둘의 차이 요약

| 항목        | Dynamic Padding | Bucketing                    |
| --------- | --------------- | ---------------------------- |
| 패딩 기준     | **각 배치의 최대 길이** | **유사한 길이끼리 묶기 때문에 패딩 거의 없음** |
| 구현 난이도    | 간단함             | 상대적으로 복잡함 (bucket 나눠야 함)     |
| 메모리 효율    | 좋음              | **더 좋음**                     |
| 데이터 순서 보존 | 가능              | 대부분 순서 섞임 (shuffle 필요)       |

---

### ✅ Hugging Face에서 적용 예시

```python
from transformers import DataCollatorWithPadding

# Dynamic Padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=data_collator,
)
```

> 🤖 `DataCollatorWithPadding`은 내부적으로 dynamic padding을 자동 처리해 줍니다.

---

원한다면 PyTorch DataLoader에서 bucketing을 수동 구현하는 방법도 알려드릴 수 있어요.
